# 实验题目 3：四阶龙格——库塔方法

## 问题分析

> 准确描述并总结出实验题目（摘要），并准确分析原题的目的和意义。

给定常微分方程初值问题：

$$
\left\{
  \begin{align*}
    \frac{dy}{dx} &= f(x,y), & a \le x \le b \\
    y(a) &= \alpha, & h = \frac{b-a}{N}
  \end{align*}
\right.
$$

求其数值解 $y_n,n=1,2,\cdots,N$。

### 实验目的

**输入：** $a,b,\alpha,N$

**输出：** 初值问题的数值解 $x_n,y_n,n=0,1,2,\cdots,N$

## 数学原理

> 数学原理表达清晰且书写准确。

记 $x_n = a + n\times h$, $n = 0,1,\cdots,N$，利用四阶龙格——库塔方法：

$$
\begin{align*}
K_1 &= hf(x_n,y_n) \\
K_2 &= hf(x_n + \frac{h}{2} + y_n + \frac{K_1}{2}) \\
K_3 &= hf(x_n + \frac{h}{2}, y_n+\frac{K_2}{2}) \\
K_4 &= hf(x_n + h, y_n + K_3) \\
y_{n+1} &= y_n + \frac{1}{6}(K_1+2K_2+2K_3+K_4) \\
&& n = 0,1,\cdots,{N-1}
\end{align*}
$$

即可逐次求出微分方程初值问题的数值解 $x_n,y_n,n=0,1,2,\cdots,N$。

## 程序设计流程

> 编译通过，根据输入能得到正确输出。

In [43]:
# 引入需要的包

from typing import *
import numpy as np
from pandas import DataFrame

In [44]:
# 四阶龙格——库塔方法
def runge_kutta(
        f: Callable[[float, float], float],
        a: float, b: float, alpha: float, N: int):
    x_list, y_list = [], []
    h = (b - a) / N
    x, y = a, alpha
    x_list.append(x)
    y_list.append(y)
    for _ in range(N):
        k_1 = h*f(x, y)
        k_2 = h*f(x+h/2, y+k_1/2)
        k_3 = h*f(x+h/2, y+k_2/2)
        k_4 = h*f(x+h, y+k_3)
        x = x + h
        y = y + (k_1 + 2*k_2 + 2*k_3 + k_4) / 6
        x_list.append(x)
        y_list.append(y)
    return x_list, y_list


In [45]:
# 运行测试参数
global_args = [
    [lambda x, y: x + y, 0, 1, -1, [5, 10, 20], lambda x: -x-1, "问题 1 (1)"],
    [lambda x, y: -y**2, 0, 1, 1, [5, 10, 20],
        lambda x: 1 / (x + 1), "问题 1 (2)"],
    [lambda x, y: 2 * y / x + x**2 +
        np.exp(x), 1, 3, 0, [5, 10, 20], lambda x: x**2 * (np.exp(x) - np.e), "问题 2 (1)"],
    [lambda x, y: (y + y**2) / x, 1, 3, -2, [5, 10, 20],
     lambda x: 2 * x / (1 - 2 * x), "问题 2 (2)"],
    [lambda x, y: -20 * (y-x*2) + 2 * x, 0, 1, 1.0 / 3,
     [5, 10, 20], lambda x: x**2 + np.exp(-20*x)/3, "问题 3 (1)"],
    [lambda x, y: -20 * y + 20 *
        np.sin(x) + np.cos(x), 0, 1, -1, [5, 10, 20], lambda x: np.exp(-20*x) + np.sin(x), "问题 3 (2)"],
    [lambda x, y: -20*(y-np.exp(x)*np.sin(x)) + np.exp(x)*(np.sin(x) + np.cos(x)),
     0, 1, 0, [5, 10, 20], lambda x: np.exp(x)*np.sin(x), "问题 3 (3)"]
]


In [46]:
# 求数据的均方误差
def get_error(f: Callable[[float], float], data):
	x, y = data
	standard = np.array([f(x_i) for x_i in x])
	return sum((y - standard) ** 2) / len(x)
	

In [47]:
# 运行一次
def run(index: int):
    res = []
    for n in global_args[index][-3]:
        data = runge_kutta(*[
            *global_args[index][:-3], n
        ])
        error = get_error(global_args[index][-2], data)
        res.append({
            "N": n,
            "标号": global_args[index][-1],
            "均方误差": error,
            "x": data[0],
            "y": data[1],
        })
    return res


In [48]:
# 运行所有并且返回结果表格
def run_all():
    all_data = [run(i) for i in range(len(global_args))]
    all = []
    for d in all_data:
        all.extend(d)
    # 重新格式化为字符串
    all = [{
            'N': d['N'],
            '标号': d['标号'],
            '均方误差': d['均方误差'],
            'x': [f"{x:.4g}" for x in d['x']],
            'y': [f"{x:.4g}" for x in d['y']],
        } for d in all]
    return DataFrame(all)


run_all()


,N,标号,均方误差,x,y
0,5,问题 1 (1),2.465190e-32,"[0, 0.2, 0.4, 0.6, 0.8, 1]","[-1, -1.2, -1.4, -1.6, -1.8, -2]"
1,10,问题 1 (1),3.182337e-31,"[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0....","[-1, -1.1, -1.2, -1.3, -1.4, -1.5, -1.6, -1.7,..."
2,20,问题 1 (1),2.206932e-31,"[0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4...","[-1, -1.05, -1.1, -1.15, -1.2, -1.25, -1.3, -1..."
3,5,问题 1 (2),2.569560e-11,"[0, 0.2, 0.4, 0.6, 0.8, 1]","[1, 0.8333, 0.7143, 0.625, 0.5556, 0.5]"
4,10,问题 1 (2),1.282857e-13,"[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0....","[1, 0.9091, 0.8333, 0.7692, 0.7143, 0.6667, 0...."
5,20,问题 1 (2),5.366889e-16,"[0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4...","[1, 0.9524, 0.9091, 0.8696, 0.8333, 0.8, 0.769..."
6,5,问题 2 (1),2.023870e+03,"[1, 1.4, 1.8, 2.2, 2.6, 3]","[0, 2.608, 8.124, 17.66, 32.55, 54.5]"
7,10,问题 2 (1),1.541237e+03,"[1, 1.2, 1.4, 1.6, 1.8, 2, 2.2, 2.4, 2.6, 2.8, 3]","[0, 1.006, 2.614, 4.947, 8.138, 12.33, 17.68, ..."
8,20,问题 2 (1),1.315425e+03,"[1, 1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1....","[0, 0.435, 1.006, 1.727, 2.614, 3.682, 4.948, ..."
9,5,问题 2 (2),7.459298e-07,"[1, 1.4, 1.8, 2.2, 2.6, 3]","[-2, -1.554, -1.384, -1.293, -1.238, -1.2]"


*为防止输出 PDF 时表格格式被破坏，在此放入上方表格的图片。*

![result_table](./imgs/lab03_result_fixed.png)

## 实验结果

> 准确规范地给出各个实验题目的结果，并对相应的思考题给出正确合理的回答与说明。

实验数据结果如上表所示。

**思考题：**

1. *对实验 1，数值解和解析解相同吗？为什么？试加以说明。*
    
    在误差范围内基本可以认为相同。由上表可知，对问题 1，当 $N = 20$ 时，其结果和标准值的均方误差均小于 $10^{-15}$，都是非常小的，所以在误差范围内可以认为数值解和解析解相同。

2. *对实验 2，N 越大越精确吗？试加以说明。*

    在实验 2 的数据中，随着 $N$ 的增大，其均方误差越来越小，所以对实验二，$N$ 越大越精确。

3. *对实验 3，N 较小会出现什么现象？试加以说明。*

    在实验 3 的数据中，当 $N$ 较小时，其均方误差非常大，达到 $10^2$ 甚至 $10^6$。